In [1]:
using TuringModels
using Turing

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/UXxKz/src/Turing.jl:24


(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);

New col contact_high, set binary values 1/0 if high/low contact

In [3]:
d[:contact_high] = map((x) -> ifelse(x=="high", 1, 0), d[:contact]);

New col where we center(!) the log_pop values

In [4]:
mean_log_pop = mean(d[:log_pop]);
d[:log_pop_c] = map((x) -> x - mean_log_pop, d[:log_pop]);

@model m10_10stan_c(total_tools, log_pop_c, contact_high) = begin
    α ~ Normal(0, 100)
    βp ~ Normal(0, 1)
    βc ~ Normal(0, 1)
    βpc ~ Normal(0, 1)

    for i ∈ 1:length(total_tools)
        λ = exp(α + βp*log_pop_c[i] + βc*contact_high[i] +
            βpc*contact_high[i]*log_pop_c[i])
        total_tools[i] ~ Poisson(λ)
    end
end;

posterior = sample(m10_10stan_c(d[:total_tools], d[:log_pop_c],
d[:contact_high]), Turing.NUTS(2000, 1000, 0.95));

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/UXxKz/src/core/ad.jl:169
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.05
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.028131021068456517, std = [1.0, 1.0, 1.0, 1.0]; 1000 iterations is used for adaption.
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/UXxKz/src/inference/adapt/adapt.jl:91
[NUTS] Finished with
  Running time        = 18.318581699000035;
  #lf / sample        = 0.0;
  #evals / sample     = 42.526;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0].


Fix the inclusion of adaptation samples

In [5]:
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names)

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Union{Missing, Float64}[3.63967 0.0 … 22.0 0.028131; 3.40297 0.0 … 22.0 0.028131; … ; 3.42866 0.0 … 94.0 0.028131; 3.09057 0.0 … 94.0 0.028131]

Rethinking result

In [6]:
m_10_10t_c_result = "
    mean   sd  5.5% 94.5% n_eff Rhat
 a   3.31 0.09  3.17  3.45  3671    1
 bp  0.26 0.03  0.21  0.32  5052    1
 bc  0.28 0.12  0.10  0.47  3383    1
 bcp 0.07 0.17 -0.20  0.34  4683    1
";

Describe the draws

In [7]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                  SD                       Naive SE                       MCSE                ESS   
       α   3.315019988  0.091735838553702195308226 0.00290094192895185573957684 0.0031938988283797039441358  824.96545
  lf_num   0.000000000  0.000000000000000000000000 0.00000000000000000000000000 0.0000000000000000000000000        NaN
      βc   0.276000210  0.118892864371637180243013 0.00375972248955857299770877 0.0033007116828740901831596 1000.00000
     βpc   0.061158820  0.177527769236720839307608 0.00561392098716809520575488 0.0052235597749018336868887 1000.00000
 elapsed   0.008183428  0.006754201446014766394010 0.00021358660345009461986945 0.0001319934568392831266120 1000.00000
 epsilon   0.028131021  0.000000000000000020827098 0.00000000000000000065861066 0.0000000000000000011564823  324.32432
      βp   0.263963144  0.037116851913097603787861 0.0011

End of m_10_10t_c.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*